In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
root_url = r"http://www.doingtime.co.uk/alphabetic-prison-listing/"

In [2]:
r = requests.get(root_url)

In [3]:
soup = BeautifulSoup(r.content, "lxml")

In [4]:
df = pd.read_html(soup.find("table").prettify(), header=0)[0]

In [5]:
counter = 0
for index, tr in enumerate(soup.find("table").findAll("tr")):
    if index == 0:
        continue
    df.loc[index-1, "url"] = tr.find("a")["href"]

In [6]:
del_index = df[df["Name"] == "Woodhill  Wormwood Scrubs"].index
df = df.drop(del_index)
df.loc[120, "Name"] = "Woodhill"
df.loc[120, "Full Name"] = "HMP Woodhill"
df.loc[120, "Region"] = "SC"
df.loc[120, "url"] = r"http://www.doingtime.co.uk/hmp-woodhill/"

df.loc[121, "Name"] = "Wormwood Scrubs"
df.loc[121, "Full Name"] = "HMP Wormwood Scrubs"
df.loc[121, "Region"] = "Lon"
df.loc[121, "url"] = r"http://www.doingtime.co.uk/hmp-wormwood-scrubs/"

In [7]:
df = df.sort("Name")

/Users/robinlinacre/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [8]:
for r in df.iterrows():
    index = r[0]
    row = r[1]
    
    url = row["url"]
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    f = soup.find("table").prettify()
    this_table = pd.read_html(f)[0]
    
    for r2 in this_table.iterrows():
        row2 = r2[1]
        col =  row2[0].lower().replace(" ", "").replace(":","")
        df.loc[index, col] = row2[1]
    

In [9]:
pd.options.display.max_columns = 100

In [10]:
df.columns = [c.lower().replace(" ","_") for c in df.columns]

In [11]:
df2 = df[["name", "full_name", "region", "url", "address", "mainswitchboard", "typeofprison"]].copy()

In [12]:
df2.columns = ["dt_name", "dt_full_name", "dt_region", "dt_url", "dt_address", "dt_main_switchboard", "dt_type_of_prison"]

In [13]:
import re
def get_postcode(address_string):
    """
    Takes an address and returns the postcode, or None if no postcode is found.
    """
    # replace strange spaces with real ones
    address_string = re.sub(r"\s|\xA0", " ", address_string)
    
    try:
        address_string = address_string.upper()
        pc_regex = "([A-PR-UWYZ]([1-9]([0-9]|[A-HJKSTUW])?|[A-HK-Y][1-9]([0-9]|[ABEHMNPRVWXY])?) *[0-9][ABD-HJLNP-UW-Z]{2}|GIR *0AA)"
        matches = re.search(pc_regex, address_string)

        if matches:
            return matches.group(1)
        else:
            return None
    except:
        return None
    
df2["dt_postcode"] = df2["dt_address"].apply(get_postcode)

In [14]:
# Get postcode and geocode :-)
import psycopg2
con_string = "host='localhost' dbname='postgres' user='postgres' password=''"
conn = psycopg2.connect(con_string)
cursor = conn.cursor()

sql = """
select
ST_X(ST_TRANSFORM(geom, 4326)) as lng,
ST_Y(ST_TRANSFORM(geom,4326)) as lat,
postcode
from all_addresses
where postcode = '{}'
"""


for r in df2.iterrows():
    row = r[1]
    index = r[0]
    
    pc = pd.read_sql(sql.format(row["dt_postcode"]),conn)
    
    if len(pc)>0:
        df2.loc[index, "dt_lat"] = pc.loc[0,"lat"]
        df2.loc[index, "dt_lng"] = pc.loc[0,"lng"]
        
        

In [19]:
df2.to_csv("doingtime_data.csv", index=False, encoding="utf-8")

In [17]:
df2

,dt_name,dt_full_name,dt_region,dt_url,dt_address,dt_main_switchboard,dt_type_of_prison,dt_postcode,dt_lat,dt_lng
0,Altcourse,HMP Altcourse,NW,http://www.doingtime.co.uk/hmp-altcourse/,Higher Lane Fazakerley Liverpool L9 7LH,0151 522 2000,Male Cat B,L9 7LH,53.460927,-2.936031
2,Ashfield,HMP Ashfield,SW,http://www.doingtime.co.uk/hmp-ashfield/,"Shortwood Road, Pucklechurch, Bristol, BS16...",0117 303 8000,Male Cat C Sex Offenders,BS16 9QJ,51.482422,-2.438866
1,Askham Grange,HMP & YOI Askham Grange,Y&H,http://www.doingtime.co.uk/hmp-yoi-askham-grange/,Askham Richard York YO23 3FT,01904 772000,Female adult and young offender,YO23 3FT,53.927081,-1.185957
3,Aylesbury,HMYOI Aylesbury,SC,http://www.doingtime.co.uk/hmyoi-aylesbury/,Bierton Road Aylesbury Bucks HP20 1EH,01296 444 000,"Male Young Offenders 18-21yrs old, with senten...",HP20 1EH,51.821980,-0.799881
4,Bedford,HMP Bedford,EoE,http://www.doingtime.co.uk/hmp-bedford/,St Loyes Street Bedford MK40 1HG,01234 373000,Local Cat B Male,MK40 1HG,52.139321,-0.470232
5,Belmarsh,HMP Belmarsh,Lon,http://www.doingtime.co.uk/hmp-belmarsh/,Western Way Thamesmead London SE28 0EB,020 8331 4400,Local male prison with high secure unit.,SE28 0EB,51.496846,0.092704
6,Birmingham,HMP Birmingham,WM,http://www.doingtime.co.uk/hmp-birmingham/,"Winson Green Road, Birmingham, B18 4AS",0121 598 8000,Male Cat B & C,B18 4AS,52.492489,-1.938513
7,Blantyre House,HMP Blantyre House,K&S,http://www.doingtime.co.uk/hmp-blantyre-house/,"Horden,Goudhurst,Kent, TN17 2NH",01580 213200,Male Cat C & D,TN17 2NH,NaN,NaN
8,Brinsford,HMYOI Brinsford,WM,http://www.doingtime.co.uk/hmp-brinsford/,"New Road, Featherstone, Wolverhampton, WV10 7PY",01902 533 450,Male Young Offenders (18-21),WV10 7PY,52.650142,-2.110816
9,Bristol,HMP Bristol,SW,http://www.doingtime.co.uk/hmp-bristol/,19 Cambridge Road Bristol BS7 8PS,0117 372 3100,Male Cat B,BS7 8PS,51.480610,-2.590665


In [16]:
a=1